In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np
from mmcv.runner import wrap_fp16_model
import random
import torch

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything()
test_indice = random.sample(range(4871), 10)

/opt/conda/envs/det/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [6]:
# from numba import cuda 
# device = cuda.get_current_device()
# device.reset()

In [40]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('/opt/ml/baseline/custom_configs/cascade_mask_rcnn_convnext/convext_large/cascade_mask_rcnn_convnext_large_fpn_giou_resize_square_relabeled.py')

root='../../dataset/'

epoch = 'epoch_36'

# dataset config 수정
cfg.data.test.test_mode = True
cfg.data.samples_per_gpu = 8
cfg.seed=42
cfg.gpu_ids = [1]
cfg.work_dir = "/opt/ml/baseline/work_dir/cc_mrcnn_convnext_large_ms_relabeled"
cfg.model.train_cfg = None

In [41]:
cfg.data.test.pipeline[1].img_scale = [(768, 768), (896, 896), (1024, 1024), (2048, 2048)]

In [42]:
cfg.model.test_cfg.rcnn.nms.iou_threshold = 0.6
cfg.model.test_cfg.rcnn.max_per_image = 1000
cfg.model.test_cfg.rcnn.score_thr = 0.0005
# cfg.model.test_cfg.rpn.nms.iou_threshold = 0.6

In [43]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [44]:
dataset.CLASSES

('Battery',
 'Clothing',
 'General_trash',
 'Glass',
 'Metal',
 'Paper',
 'Paper_pack',
 'Plastic',
 'Plastic_bag',
 'Styrofoam')

In [45]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
wrap_fp16_model(model)
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

load checkpoint from local path: /opt/ml/baseline/work_dir/cc_mrcnn_convnext_large_ms_relabeled/epoch_36.pth


In [46]:
output = single_gpu_test(model, data_loader, show_score_thr=0.0005) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 10.3 task/s, elapsed: 472s, ETA:     0s

In [14]:
eval_result = dataset.evaluate(output)


Evaluating bbox...
Loading and preparing results...
DONE (t=0.43s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=6.93s).
Accumulating evaluation results...
DONE (t=1.86s).

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = -1.000
 Average Recall     (AR) @[ IoU=

In [15]:
eval_result

OrderedDict([('bbox_mAP', -1.0),
             ('bbox_mAP_50', -1.0),
             ('bbox_mAP_75', -1.0),
             ('bbox_mAP_s', -1.0),
             ('bbox_mAP_m', -1.0),
             ('bbox_mAP_l', -1.0),
             ('bbox_mAP_copypaste',
              '-1.000 -1.000 -1.000 -1.000 -1.000 -1.000')])

In [50]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'cc_mrcnn_convnext_large_fpn_giou_relabeled_TTA_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [23]:
submission

,PredictionString,image_id
0,0 0.014751938 371.52756 620.98773 391.3011 641...,test/0000.jpg
1,-1 0.0008013897 350.84436 249.5179 752.3004 69...,test/0001.jpg
2,-1 0.026546903 753.84314 235.70558 919.6193 32...,test/0002.jpg
3,8 0.998132 143.63788 263.73062 928.0228 814.30...,test/0003.jpg
4,-1 0.010676341 459.5907 409.808 660.946 551.17...,test/0004.jpg
...,...,...
4866,0 0.005335262 297.39987 346.13263 672.7805 688...,test/4866.jpg
4867,9 0.0043957303 610.3399 320.3145 781.1391 494....,test/4867.jpg
4868,9 0.0005019565 444.22235 455.38513 755.84985 8...,test/4868.jpg
4869,8 0.001668307 81.015396 499.18497 583.97754 82...,test/4869.jpg


In [15]:
COLORS = [tuple(map(int, np.random.choice(range(256), size=3))) for i in range(len(classes))]

In [55]:
import cv2
# submission 양식에 맞게 output 후처리

test_vis_path = os.path.join(cfg.work_dir, "test_images_tta")
if not os.path.exists(test_vis_path):
    os.mkdir(test_vis_path)


prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i in test_indice:
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    image_path = os.path.join(root, image_info['file_name'])
    img = cv2.imread(image_path)
    for j in range(class_num):
        for o in output[i][j]:
            boxes = list(map(int, o[:4]))
            cv2.rectangle(img, (boxes[0], boxes[1]), (boxes[2], boxes[3]), COLORS[j], 1)
            cv2.putText(img, f"{classes[j]}", (boxes[0], boxes[1]-5), fontFace=0, fontScale=1, color=COLORS[j], thickness=3)
            cv2.imwrite(os.path.join(test_vis_path, f"{image_info['id']}.jpg"), img)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [69]:
image_info

{'width': 1024,
 'height': 1024,
 'file_name': 'test/4238.jpg',
 'license': 0,
 'flickr_url': None,
 'coco_url': None,
 'date_captured': '2021-01-04 15:01:49',
 'id': 4238}

In [93]:
coco = COCO("/opt/ml/dataset/split_valid.json")

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [94]:
import shutil
img_ids = coco.getImgIds()
for id in img_ids:
    info = coco.loadImgs(id)[0]
    src = os.path.join(root, info['file_name'])
    dst = os.path.join("/opt/ml/dataset/images/valid", info['file_name'].split('/')[-1])
    shutil.copy(src, dst)
